In [1]:
# 웹크롤링용 라이브러리
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

# 기타 라이브러리
import time
import warnings
warnings.filterwarnings('ignore')

# 데이터프레임용 라이브러리
import pandas as pd
# 위도경도 변환용 라이브러리
#!pip install geopy
from geopy.geocoders import GoogleV3

In [2]:
def check_element_exists(css_selector):
  try:
    test = driver.find_element(By.CSS_SELECTOR, css_selector)
    #print('false')
    return False
  except NoSuchElementException:
    #print('true')
    return True

In [3]:
url = 'https://map.forest.go.kr/forest/'
# Google Geolocator 객체 생성
geolocator = GoogleV3(api_key="본인 API 키 사용")

options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome()
driver.get(url)

choso = []

btn = driver.find_element(By.CSS_SELECTOR, '#cookiecheck')
btn.click()
time.sleep(0.5)

search = driver.find_element(By.CSS_SELECTOR, '#poisearch-text').send_keys('산불감시초소')
time.sleep(0.5)
search_btn = driver.find_element(By.CSS_SELECTOR, '#poi-search-btn > input[type=image]')
search_btn.click()
time.sleep(0.5)

while True : 
  css_selector = "#result-table_wrapper > div:nth-child(3) > div:nth-child(2) > div > ul > li.next.disabled"
  is_exists = check_element_exists(css_selector)

  # 다음 버튼이 비활성화 될 경우 반복 중지
  if is_exists == False :
    break
  
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  infos = soup.select('#dorojusoResultTbody > tr')
  
  for info in infos :
    # 초소명
    name = info.select_one('#dorojusoResultTbody > tr > td:nth-child(1)').text
    address = info.select_one('#dorojusoResultTbody > tr > td:nth-child(2)').text

    sido = address.split(' ')[0]
    
    try:
      # 주소를 위도와 경도로 변환
      location = geolocator.geocode(address, timeout=10)
      if location:
        print(f"위도: {location.latitude}, 경도: {location.longitude}")
        latitude = location.latitude
        longitude = location.longitude
      else:
        print("주소를 변환할 수 없습니다.")
    except Exception as e:
      print(f"오류 발생: {e}")
      latitude = 0
      longitude = 0
    time.sleep(0.3)
    choso.append([name, address, sido, latitude, longitude])
  
  if is_exists == True :
    next_btn = driver.find_element(By.CSS_SELECTOR, '#result-table_wrapper > div:nth-child(3) > div:nth-child(2) > div > ul > li.next > a').click()
    time.sleep(0.5)
    
  # 다음 버튼이 비활성화 될 경우 반복 중지
  else : 
    break
driver.quit()

choso_df = pd.DataFrame(choso, columns = ['name', 'address', 'sido', 'latitude', 'longitude'])
choso_df.info()
choso_df

오류 발생: Non-successful status code 400
위도: 37.6440788, 경도: 128.8583955
위도: 37.2939649, 경도: 129.0531631
위도: 37.2941049, 경도: 129.0578108
위도: 37.2546344, 경도: 129.0288323
위도: 37.4417063, 경도: 129.0806904
위도: 37.166068, 경도: 128.975762
위도: 37.4622371, 경도: 127.0010996
위도: 37.8761775, 경도: 127.1085076
위도: 37.4417117, 경도: 127.1194117
위도: 37.3108855, 경도: 127.04395
위도: 37.4867975, 경도: 127.2278472
위도: 34.97212830000001, 경도: 128.0778743
위도: 35.7652373, 경도: 128.7098336
위도: 35.8173, 경도: 128.7249203
위도: 35.7558531, 경도: 129.1902154
위도: 35.7688789, 경도: 129.1643648
위도: 35.9197219, 경도: 129.2336126
위도: 35.9228168, 경도: 129.2465676
위도: 35.77946439999999, 경도: 128.6310007
위도: 35.7728472, 경도: 128.6742491
위도: 35.8148038, 경도: 128.6503842
위도: 35.81515220000001, 경도: 128.6501165
위도: 35.8136968, 경도: 128.6451793
위도: 35.817251, 경도: 128.635009
위도: 35.1107428, 경도: 128.8574527
위도: 35.1107428, 경도: 128.8574527
위도: 35.2420878, 경도: 129.2294175
위도: 35.180513, 경도: 129.0438551
위도: 35.5492363, 경도: 129.3859589
위도: 36.5146151, 경도: 127

,name,address,sido,latitude,longitude
0,산불감시초소,,,0.000000,0.000000
1,산불감시초소,강원특별자치도 강릉시 왕산면 목계리 459-1,강원특별자치도,37.644079,128.858396
2,산불감시초소,강원특별자치도 삼척시 도계읍 발이리 323-4,강원특별자치도,37.293965,129.053163
3,산불감시초소,강원특별자치도 삼척시 도계읍 발이리 439,강원특별자치도,37.294105,129.057811
4,산불감시초소,강원특별자치도 삼척시 도계읍 점리 154,강원특별자치도,37.254634,129.028832
5,산불감시초소,강원특별자치도 삼척시 미로면 내미로리 44-1,강원특별자치도,37.441706,129.080690
6,산불감시초소,강원특별자치도 태백시 황지동 630,강원특별자치도,37.166068,128.975762
7,산불감시초소,경기도 과천시 과천동 112-8,경기도,37.462237,127.001100
8,산불감시초소,경기도 동두천시 탑동동 587-3,경기도,37.876177,127.108508
9,산불감시초소,경기도 성남시 수정구 태평동 7003-20천,경기도,37.441712,127.119412


In [4]:
choso_df = choso_df.drop(0).reset_index(drop=True)
choso_df.to_csv('초소이름.csv', encoding = 'utf-8', index = False)